### Download the code zip file

In [1]:
!wget https://pyimagesearch-code-downloads.s3.us-west-2.amazonaws.com/pytorch-object-detection/pytorch-object-detection.zip
!unzip -qq pytorch-object-detection.zip
%cd pytorch-object-detection

--2023-09-22 17:19:41--  https://pyimagesearch-code-downloads.s3.us-west-2.amazonaws.com/pytorch-object-detection/pytorch-object-detection.zip
Resolving pyimagesearch-code-downloads.s3.us-west-2.amazonaws.com (pyimagesearch-code-downloads.s3.us-west-2.amazonaws.com)... 3.5.86.73, 52.218.180.137, 3.5.78.102, ...
Connecting to pyimagesearch-code-downloads.s3.us-west-2.amazonaws.com (pyimagesearch-code-downloads.s3.us-west-2.amazonaws.com)|3.5.86.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 401785 (392K) [application/zip]
Saving to: ‘pytorch-object-detection.zip’

pytorch-object-dete 100%[===================>] 392.37K   446KB/s    in 0.9s    

2023-09-22 17:19:42 (446 KB/s) - ‘pytorch-object-detection.zip’ saved [401785/401785]

/content/pytorch-object-detection


### Import Packages

In [2]:
!gdown "https://drive.google.com/u/1/uc?id=1H-c_LQY5qA7YyksffRuPDr4SVzY_-Xp0&export=download"

Downloading...
From: https://drive.google.com/u/1/uc?id=1H-c_LQY5qA7YyksffRuPDr4SVzY_-Xp0&export=download
To: /content/pytorch-object-detection/yolo dataset3.0.zip
100% 384M/384M [00:05<00:00, 66.6MB/s]


In [3]:
!unzip "/content/pytorch-object-detection.zip" -d "/content/pytorch-object-detection/images"

Archive:  /content/pytorch-object-detection.zip
   creating: /content/pytorch-object-detection/images/pytorch-object-detection/
   creating: /content/pytorch-object-detection/images/pytorch-object-detection/images/
 extracting: /content/pytorch-object-detection/images/pytorch-object-detection/detect_image.py  
 extracting: /content/pytorch-object-detection/images/pytorch-object-detection/detect_realtime.py  
 extracting: /content/pytorch-object-detection/images/pytorch-object-detection/coco_classes.pickle  
 extracting: /content/pytorch-object-detection/images/pytorch-object-detection/images/example_01.jpg  
 extracting: /content/pytorch-object-detection/images/pytorch-object-detection/images/example_05.jpg  
 extracting: /content/pytorch-object-detection/images/pytorch-object-detection/images/example_06.jpg  
 extracting: /content/pytorch-object-detection/images/pytorch-object-detection/images/example_04.jpg  
 extracting: /content/pytorch-object-detection/images/pytorch-object-detect

In [4]:
datapath = "/content/pytorch-object-detection/images/pytorch-object-detection/images"

In [6]:
pip install ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.0/618.0 kB 11.2 MB/s eta 0:00:00


In [7]:
# Some other imports
import shutil
import cv2
from google.colab.patches import cv2_imshow # cv2.imshow work around for colab
from ultralytics.utils.plotting import Annotator # For annotating result images

In [14]:
from torchvision.models import detection
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import torch
import cv2
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection import FasterRCNN
from torchvision.transforms import functional as F
from torchvision.models.detection.rpn import AnchorGenerator


In [11]:
# Load a pre-trained model
model = fasterrcnn_resnet50_fpn(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:02<00:00, 77.2MB/s]


In [27]:
print("Pretrained Model Architecture:")
print(model)

Pretrained Model Architecture:
FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, e

In [28]:
# Create a custom Faster R-CNN model with ResNet-50 backbone from scratch
class CustomFasterRCNN(torch.nn.Module):
    def __init__(self, num_classes):
        super(CustomFasterRCNN, self).__init__()
        # Load a pre-trained ResNet-50 model as the backbone
        self.backbone = torchvision.models.resnet50(pretrained=True)
        # Modify the output channels of the backbone
        self.backbone.out_channels = 256

        # Create an anchor generator
        rpn_anchor_generator = AnchorGenerator(
            sizes=((32, 64, 128, 256, 512),),
            aspect_ratios=((0.5, 1.0, 2.0),)
        )

        # Define the Region Proposal Network (RPN)
        rpn_head = torchvision.models.detection.rpn.RPNHead(
            self.backbone.out_channels,
            rpn_anchor_generator.num_anchors_per_location()[0]
        )

        # Create the Faster R-CNN model
        self.model = FasterRCNN(
            self.backbone,
            num_classes=num_classes,  # Number of object classes in your dataset
            rpn_anchor_generator=rpn_anchor_generator,
            rpn_head=rpn_head
        )

    def forward(self, images, targets=None):
        if self.training and targets is None:
            raise ValueError("In training mode, targets should be passed to the forward method.")
        return self.model(images, targets)


In [30]:
# Create an instance of the custom model with the same number of classes as the pretrained model
num_classes = 10  # For COCO dataset
custom_model = CustomFasterRCNN(num_classes)


In [31]:
# Print the architecture of the custom model
print("\nCustom Model Architecture:")
print(custom_model)


Custom Model Architecture:
CustomFasterRCNN(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      

In [32]:
# Save the custom model to a file (optional)
torch.save(custom_model.state_dict(), "custom_model.pth")

In [36]:
model = ("/content/pytorch-object-detection/custom_model.pth")

In [37]:
pickle_file = "/content/pytorch-object-detection/coco_classes.pickle"
results = model.train( data= pickle_file, epochs=100)

AttributeError: ignored